# Energy calibration
このノートでは、検出器のraw ADCの値をch毎に線形関数でキャリブレーションし、エネルギー情報に変換する。
キャリブレーションパラメータはCSVファイルで準備済みとする。

- data/clover_data.parquet:
こちらは、クローバー型ゲルマニウム検出器2台からなるデータで、各検出器に4つの読み出しチャンネルがあり、
合計8chとなっている。

- ClvCalib.csv: キャリブレーションパラメータ

## ファイルを開く

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()
df = spark.read.parquet("./data/clover_data.parquet")
df_select = df.select("ts","cfd","ch","adc")
df_select.show(5)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/08 17:14:54 WARN Utils: Your hostname, TABLET-S9I8ER9S, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/08 17:14:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/08 17:14:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+-------------+-----+---+----+
|           ts|  cfd| ch| adc|
+-------------+-----+---+----+
|7296321907778| 8485|  4|3982|
|7296321978243| 9278|  2| 774|
|7296321997358| 8225|  1|1760|
|7296322069363|16234|  0|   0|
|7296322214400|24255|  4|4526|
+-------------+-----+---+----+
only showing top 5 rows


とりあえず"ch"と"adc"列だけセレクト
## 整数値を実数値に変換する。
raw ADCの値は整数値なので、0から1の乱数を足して実数値にする。

In [2]:
df_real = df_select.withColumn("adc_real", F.col("adc")+F.rand())
df_real.show(5)

+-------------+-----+---+----+------------------+
|           ts|  cfd| ch| adc|          adc_real|
+-------------+-----+---+----+------------------+
|7296321907778| 8485|  4|3982| 3982.394205589545|
|7296321978243| 9278|  2| 774| 774.3927821492072|
|7296321997358| 8225|  1|1760|1760.7673300781569|
|7296322069363|16234|  0|   0|0.8418421262645442|
|7296322214400|24255|  4|4526| 4526.628214969451|
+-------------+-----+---+----+------------------+
only showing top 5 rows


## CSVからキャリブレーションパラメータのデータフレームを作る
CSVファイルがあるとする。
```
ch,p0,p1
0,0.2149837759,0.0803461538
1,0.1280723048,0.2546962000
2,-0.1933825665,0.2941007359
3,0.0611566264,0.0929934353
4,-0.0563791084,0.2946970847
5,0.0371477634, 0.1212543693
6,-0.3015558027966563,0.29948038289716816
7,-0.09316673031071332,0.28713228807294117
```
spark.read.csv() でCSVからデータフレームを作成。
`header=True`は一行目を列の名前として読むオプション。
`inferSchema=True`はデータをstringではなく、推論した型で読み込むオプション。

In [3]:
param_df = spark.read.csv("./ClvCalib.csv", header=True, inferSchema=True)
param_df.show()

+---+--------------------+-------------------+
| ch|                  p0|                 p1|
+---+--------------------+-------------------+
|  0|        0.2149837759|       0.0803461538|
|  1|        0.1280723048|          0.2546962|
|  2|       -0.1933825665|       0.2941007359|
|  3|        0.0611566264|       0.0929934353|
|  4|       -0.0563791084|       0.2946970847|
|  5|        0.0371477634|       0.1212543693|
|  6| -0.3015558027966563|0.29948038289716816|
|  7|-0.09316673031071332|0.28713228807294117|
+---+--------------------+-------------------+



## ch番号が一致するパラメータを元のデータフレームにくっつける。
- join(): 二つのデータフレームを`on=`で指定した条件を満たす行同士で結合する。`how="inner"`は両方のデータフレームの行が存在するものだけ出力、`how="outer"`は片方が存在しない行も残す。

In [4]:
calib_df = df_real.join(param_df, on=["ch"], how="inner")
calib_df.show(5)

+---+-------------+-----+----+------------------+-------------+------------+
| ch|           ts|  cfd| adc|          adc_real|           p0|          p1|
+---+-------------+-----+----+------------------+-------------+------------+
|  4|7296321907778| 8485|3982| 3982.394205589545|-0.0563791084|0.2946970847|
|  2|7296321978243| 9278| 774| 774.3927821492072|-0.1933825665|0.2941007359|
|  1|7296321997358| 8225|1760|1760.7673300781569| 0.1280723048|   0.2546962|
|  0|7296322069363|16234|   0|0.8418421262645442| 0.2149837759|0.0803461538|
|  4|7296322214400|24255|4526| 4526.628214969451|-0.0563791084|0.2946970847|
+---+-------------+-----+----+------------------+-------------+------------+
only showing top 5 rows


p0、p1の列に、ch番号に対応する値が入った。
## パラメータを使ってエネルギーに変換
ここまでできればあとは新しくadc_calを定義するだけ。

In [5]:
calib_df = calib_df.withColumn("adc_cal", F.expr("p0 + adc_real * p1"))
calib_df.show(5)

+---+-------------+-----+----+------------------+-------------+------------+------------------+
| ch|           ts|  cfd| adc|          adc_real|           p0|          p1|           adc_cal|
+---+-------------+-----+----+------------------+-------------+------------+------------------+
|  4|7296321907778| 8485|3982| 3982.394205589545|-0.0563791084|0.2946970847|1173.5435834050113|
|  2|7296321978243| 9278| 774| 774.3927821492072|-0.1933825665|0.2941007359|227.55610453923023|
|  1|7296321997358| 8225|1760|1760.7673300781569| 0.1280723048|   0.2546962|448.58882035985226|
|  0|7296322069363|16234|   0|0.8418421262645442| 0.2149837759|0.0803461538|0.2826225528521701|
|  4|7296322214400|24255|4526| 4526.628214969451|-0.0563791084|0.2946970847| 1333.927759363862|
+---+-------------+-----+----+------------------+-------------+------------+------------------+
only showing top 5 rows


## ファイルに書き出す
- write.parquet(): parquetファイルにデータフレームを書き出す。
- mode("overwrite"): ファイルが存在する場合、上書きする。

In [6]:
calib_df.select("ts","ch","cfd","adc_cal").write.mode("overwrite").parquet("./data/clover_caldata.parquet")